In [1]:
pip install kalshi-python

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
import kalshi_python
from pprint import pprint
import numpy as np
# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid
from kalshi_python.models import *
from datetime import datetime

In [4]:
# To start off, you need to have created an account at https://kalshi.com (Production) 
# or an account on the Demo https://demo.kalshi.co/

# prod_email = "" # change these to be your personal credentials
# prod_password = "" # (for extra security, we recommend using a config file)

demo_email = "adk1361@bu.edu" # change these to be your personal credentials
demo_password = "Cascs542@123" # (for extra security, we recommend using a config file)

# for prod
# prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in prod:
# exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

## if wanting to test in demo
exchange_client = ExchangeClient(exchange_api_base = demo_api_base, email = demo_email, password = demo_password)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': True}


In [5]:
config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = 'https://demo-api.kalshi.co/trade-api/v2'

# Create an API configuration passing your credentials.
# Use this if you want the kalshi_python sdk to manage the authentication for you.
kalshi_api = kalshi_python.ApiInstance(
    email='sgoyal15@bu.edu',
    password='Welcome123@',
    configuration=config,
)

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
print(exchangeStatus)

Exchange status response: 
{'exchange_active': True, 'trading_active': True}


In [7]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.5 MB 3.3 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/11.5 MB 8.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/11.5 MB 21.3 MB/s eta 0:00:01
   -------------------- ------------------- 5.9/11.5 MB 34.2 MB/s eta 0:00:01
   ------------------------------ --------- 8.7/11.5 MB 42.7 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.5 MB 59.8 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 50.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/505.5 kB ? eta -:--:--
   ---------------------------------------- 505.5/505.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/345.4 kB ? eta -:--:--
   --------------------------------------- 345.4/345.4 kB 20.9 MB/s eta 0:00:00
Note: you m

In [9]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('data_for_tradingapi.csv')
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)
# print(df)

df['predicted_tempmax'] = np.round(df['predicted_tempmax'],0)
df['predicted_tempmax'] += 3
print(df)

# # Get the predicted temperature for Austin
# predicted_temp_for_austin = df.loc[df['Location'] == 'Austin', 'predicted_tempmax'].values[0]
# print(f"Predicted Temperature for Austin: {predicted_temp_for_austin}")


   Location  predicted_tempmax
0    Austin               86.0
1   Chicago               60.0
2     Miami               80.0
3  New York               63.0


In [10]:
def determine_market_ticker_from_response(eventResponse, predicted_max_temp):
    for market in eventResponse.markets:
        subtitle = market.subtitle  # Adjusted based on your object structure
        if 'or below' in subtitle:
            max_temp = int(subtitle.split('°')[0])
            if predicted_max_temp <= max_temp:
                return market.ticker.split('-')[-1]
        elif 'or above' in subtitle:
            min_temp = int(subtitle.split('°')[0]) + 1
            if predicted_max_temp >= min_temp:
                return market.ticker.split('-')[-1]
        else:
            # Handling ranges correctly by cleaning the string of non-numeric before conversion
            temp_range = [int(temp.split('°')[0]) for temp in subtitle.split(' to ')]
            if temp_range[0] <= predicted_max_temp <= temp_range[1]:
                return market.ticker.split('-')[-1]
    return None


In [12]:
seriesTickers = {
    'New York': 'HIGHNY',
    'Chicago': 'HIGHCHI',
    'Austin': 'HIGHAUS',
    'Miami': 'HIGHMIA'
}
today = datetime.today()

# print(f"Processing cities: {df['Location'].tolist()}")

# for index, row in df.iterrows():
#     city = row['Location']
#     print(f"Processing {city}...")

# # Format the date as 'YYMMDD'
date_str = today.strftime('%y%b%d').upper()
for index, row in df.iterrows():
    city = row['Location']
    predicted_max_temp = row['predicted_tempmax']
    seriesTicker = seriesTickers[city]
    

    eventTicker = f'{seriesTicker}-{date_str}'
    eventResponse = kalshi_api.get_event(eventTicker)  # Replace with actual API call
    
    # Parse the event response to find matching market ticker
    # This code block assumes a function to parse and match the market ticker based on predicted temperature
    matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
    # Now you have the matching market ticker, you can retrieve the order book
    marketTicker = f'{eventTicker}-{matching_market_ticker}'
    orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)  # Replace with actual API call
    
    print(f'\nOrderbook for market: {marketTicker}')
    # Assuming pprint is imported or defined
    pprint(orderbookResponse)
# print(dir(orderbookResponse))



Orderbook for market: HIGHAUS-24APR04-B86.5
{'orderbook': {'no': [], 'yes': [[99, 1]]}}

Orderbook for market: HIGHCHI-24APR04-T46
{'orderbook': {'no': [], 'yes': [[99, 1]]}}

Orderbook for market: HIGHMIA-24APR04-B80.5
{'orderbook': {'no': [], 'yes': [[99, 1]]}}

Orderbook for market: HIGHNY-24APR04-T54
{'orderbook': {'no': [], 'yes': [[99, 1]]}}


In [13]:
yes_offers = orderbookResponse.orderbook.yes
orderbook_dict = orderbookResponse.to_dict()
yes_offers = orderbook_dict['orderbook']['yes']


In [15]:
orderbook_dict = orderbookResponse.to_dict()
yes_offers = orderbook_dict['orderbook']['yes']
for index, row in df.iterrows():
    city = row['Location']
    predicted_max_temp = row['predicted_tempmax']
    seriesTicker = seriesTickers[city]

    eventTicker = f'{seriesTicker}-{date_str}'
    eventResponse = kalshi_api.get_event(eventTicker)

    matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
    if matching_market_ticker:
        marketTicker = f'{eventTicker}-{matching_market_ticker}'
        orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
        
        # Convert the orderbookResponse to a dictionary to access 'yes' and 'no' offers
        orderbook_dict = orderbookResponse.to_dict()
        yes_offers = orderbook_dict['orderbook']['yes'] if 'yes' in orderbook_dict['orderbook'] else []
        
        if yes_offers:
            best_yes_offer = min([offer[0] for offer in yes_offers])
            
            if exchangeStatus.trading_active:
                orderUuid = str(uuid.uuid4())
                orderResponse = kalshi_api.create_order(CreateOrderRequest(       
                    ticker=marketTicker,
                    action='buy',
                    type='limit',
                    yes_price=best_yes_offer - 1,  # Adjust based on your strategy
                    count=1000,  # And your desired amount
                    client_order_id=orderUuid,
                    side='yes',
                ))
                print(f'\nOrder submitted for {city}: ')
                pprint(orderResponse)
        else:
            print(f'\nNo "yes" offers available for {city}, no orders will be placed.')
    else:
        print(f'No matching market ticker found for {city} based on the predicted temperature.')
 


Order submitted for Austin: 
{'order': {'action': 'buy',
           'client_order_id': '730d9fdf-edb9-408c-8905-1036e5924951',
           'created_time': '2024-04-05T01:18:29.307094Z',
           'expiration_time': None,
           'no_price': 2,
           'order_id': '09fbfa10-0fad-4a88-baa3-cf83c3be74e6',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'HIGHAUS-24APR04-B86.5',
           'type': 'limit',
           'user_id': 'dbad063d-b115-48bd-a024-079c02ba5ad4',
           'yes_price': 98}}

Order submitted for Chicago: 
{'order': {'action': 'buy',
           'client_order_id': '84d6e1e5-0050-49f0-a098-a6425436e437',
           'created_time': '2024-04-05T01:18:29.633335Z',
           'expiration_time': None,
           'no_price': 2,
           'order_id': 'b60cc942-f173-417d-abce-0b0dae092beb',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'HIGHCHI-24APR04-T46',
           'type': 'limit',
           'user_id': 'd